# Анализ сайта gismeteo.ru

Для выполнения итогового задания в качестве источника данных метеосводок был выбран сервис Дневник сайта Gismeteo.ru, в котором можно бесплатно получить исторические данные о погоде, начиная с 1997 года. Необходимо получить данные с 2008 по 2018 года включительно (последний год необходим для прогнозирования навигации на сезон).

todo: добавить список коммерческих ресурсов, с которых можно получить исторические данные и прогнозы на долгий срок

![Внешний вид сервиса Дневник от gismeteo](images/gismeteo_example.png)

После исполнения скрипта на загрузку данных из сайта автоматизированной информационной системы государственного мониторинга водных объектов Российской Федерации (далее - "АИС ГМВО") **src\preparation\scrape_water_level.py**, в **data\raw\water_posts_data.json** будет находиться словарь **{uid: наименование поста наблюдения}**. Наименования представлены в виде: *название водного объекта* - *название населённого пункта*

## Получение списка населённых пунктов с постами наблюдения

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка списка постов

In [28]:
from src.utils import *

if not is_data_exists(DATA_POSTS_RAW, is_raw=True):
    print('Данные с постами наблюдения АИС ГМВО не были загружены. Пожалуйста, '
          'выполните скрипт src\preparation\scrape_water_level.py'
          'и попробуйте ещё раз.')
else:
    file_posts = open_file(DATA_POSTS_RAW, is_raw=True)
    posts = json.loads(file_posts)
print(len(posts))
posts

28


{'09386': 'р.Подкаменная Тунгуска - пос.Чемдальск',
 '09387': 'р.Подкаменная Тунгуска - с.Ванавара',
 '09388': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
 '09389': 'р.Подкаменная Тунгуска - с.Байкит',
 '09390': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
 '09392': 'р.Чуня - пос.Стрелка Чуня',
 '09393': 'р.Чуня - пос.Муторай',
 '09396': 'руч.Миханский - пос.Вельмо 2-е',
 '09397': 'р.Тея - пгт Тея',
 '09499': 'р.Бурная - пос.Бурный',
 '09523': 'р.Подкаменная Тунгуска - пос.Суломай',
 '09560': 'р.Вельмо - пос.Светлана',
 '09568': 'р.Вельмо - пос.Вельмо 2-е',
 '09403': 'р.Нижняя Тунгуска - д.Верхнекарелина',
 '09404': 'р.Нижняя Тунгуска - с.Подволошино',
 '09405': 'р.Нижняя Тунгуска - с.Преображенка',
 '09406': 'р.Нижняя Тунгуска - с.Ербогачен',
 '09408': 'р.Нижняя Тунгуска - пос.Кислокан',
 '09410': 'р.Нижняя Тунгуска - пгт Тура',
 '09413': 'р.Нижняя Тунгуска - пос.Учами',
 '09415': 'р.Нижняя Тунгуска - факт.Большой Порог',
 '09416': 'р.Непа - с.Токма',
 '09417': 'р.Непа - д.Ика',
 '09419'

### Создание набора с очищенными названиями населённых пунктов

In [29]:
#test_name = next(iter(posts.values()))  # р.Подкаменная Тунгуска - пос.Чемдальск
#test_name = posts['09388']  # р.Подкаменная Тунгуска - факт.Усть-Камо
test_name = posts['09415']  # р.Нижняя Тунгуска - факт.Большой Порог
test_name

'р.Нижняя Тунгуска - факт.Большой Порог'

In [31]:
import re

'''
Между видом и названием населённого пункта есть или точка, или пробел, используем [ \.]{1}
Учтено наличие в названии пункта пробела и тире
В БД gismeteo названия некоторых пунктов отличаются от таковых из сайта АИС ГМВО
'''
def clear_name(input_str):
    result = input_str.replace('Вельмо 2-е', 'Вельмо')
    result = result.replace('Стрелка Чуня', 'Стрелка-Чуня')
    result = result.replace('Большой порог', 'Большой Порог')
    result = re.search(r" - \w+[ \.]{1}([\w -]+)", result).group(1)
    return result

clear_name(test_name)

'Большой Порог'

In [32]:
for uid, name in posts.items():
    posts[uid] = {'name': name, 'clean_name': clear_name(name)}

posts

{'09386': {'name': 'р.Подкаменная Тунгуска - пос.Чемдальск',
  'clean_name': 'Чемдальск'},
 '09387': {'name': 'р.Подкаменная Тунгуска - с.Ванавара',
  'clean_name': 'Ванавара'},
 '09388': {'name': 'р.Подкаменная Тунгуска - факт.Усть-Камо',
  'clean_name': 'Усть-Камо'},
 '09389': {'name': 'р.Подкаменная Тунгуска - с.Байкит',
  'clean_name': 'Байкит'},
 '09390': {'name': 'р.Подкаменная Тунгуска - факт.Кузьмовка',
  'clean_name': 'Кузьмовка'},
 '09392': {'name': 'р.Чуня - пос.Стрелка Чуня', 'clean_name': 'Стрелка-Чуня'},
 '09393': {'name': 'р.Чуня - пос.Муторай', 'clean_name': 'Муторай'},
 '09396': {'name': 'руч.Миханский - пос.Вельмо 2-е', 'clean_name': 'Вельмо'},
 '09397': {'name': 'р.Тея - пгт Тея', 'clean_name': 'Тея'},
 '09499': {'name': 'р.Бурная - пос.Бурный', 'clean_name': 'Бурный'},
 '09523': {'name': 'р.Подкаменная Тунгуска - пос.Суломай',
  'clean_name': 'Суломай'},
 '09560': {'name': 'р.Вельмо - пос.Светлана', 'clean_name': 'Светлана'},
 '09568': {'name': 'р.Вельмо - пос.Вельм

### Создание списка с уникальными населёнными пунктами

In [33]:
names = set([x['clean_name'] for x in posts.values()])
names

{'Байкит',
 'Большой Порог',
 'Бурный',
 'Ванавара',
 'Вельмо',
 'Верхнекарелина',
 'Ербогачен',
 'Ика',
 'Кербо',
 'Кислокан',
 'Кузьмовка',
 'Муторай',
 'Непа',
 'Подволошино',
 'Преображенка',
 'Светлана',
 'Стрелка-Чуня',
 'Суломай',
 'Тембенчи',
 'Тея',
 'Токма',
 'Тура',
 'Усть-Камо',
 'Учами',
 'Чемдальск'}

In [34]:
print(f'Число постов: {len(posts)}\nЧисло уникальных пунктов: {len(names)}')

Число постов: 28
Число уникальных пунктов: 25


## Проверка наличия прогноза погоды

С получением метео-данных есть 2 проблемы:
1. Не все населённые пункты представлены в списке городов.
2. В некоторых пунтах за период с 2008 по 2018 не велись наблюдения метео-данных. При попытке их получить сайт показывает сообщение:
> Наблюдения метео-данных в данный период не велись.

![Сообщение об ошибке](images/gismeteo_error.png)

Необходимо обнаружить проблемные населённые пункты.

### Получение списка населённых пунктов с gismeteo

После открытия https://www.gismeteo.ru/diary/ и выбора страны Россия и области Красноярский край происходит 3 GET запроса:
![GET запросы](images/gismeteo_requests.png)

1. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/countries/?fr=sel  
Получение списка стран c ID и названиями.

2. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/districts/?country=156&fr=sel  
Получения списка областей c ID и названиями. В параметре country указан ID выбранной страны (156 - Россия).

3. https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel  
Получения списка городов c ID и названиями. В параметре district указан ID выбранной области (301 - Красноярский край).

Во всех запросах **63466572668a39754a9ddf4c8b3437b0** - это стандартный api-токен, который не зависит от IP-адреса пользователя. Запросы возвращают данные в формате xml.

In [9]:
from src.utils import get_url
import xmltodict

if is_data_exists(DATA_GISMETEO_CITIES, is_raw=True):
    print('Используем ранее загруженные данные о городах')
    cities = open_file(DATA_GISMETEO_CITIES, is_raw=True)
else:
    url = 'https://www.gismeteo.ru/inform-service/63466572668a39754a9ddf4c8b3437b0/cities/?district=301&fr=sel'
    r = get_url(url)
    cities = r.text
    write_data(DATA_GISMETEO_CITIES, data=cities, is_raw=True)

cities = xmltodict.parse(cities)  # конвертация xml в dict

# основные данные находятся в cities['document']['item']
cities = cities['document']['item']
cities[0:2]

Используем ранее загруженные данные о городах


[OrderedDict([('@id', '157182'),
              ('@n', 'Абакумовка'),
              ('@country_id', '156'),
              ('@country_name', 'Россия'),
              ('@district_id', '301'),
              ('@district_name', 'Красноярский край'),
              ('@kind', 'T')]),
 OrderedDict([('@id', '157882'),
              ('@n', 'Абакшино'),
              ('@country_id', '156'),
              ('@country_name', 'Россия'),
              ('@district_id', '301'),
              ('@district_name', 'Красноярский край'),
              ('@kind', 'T')])]

In [10]:
# создадим новый словарь {id: название} на основе полученных данных
cities_gismeteo = []

for city in cities:
    city_id = city['@id']
    name = city['@n']
    if name in names:
        cities_gismeteo.append({'city_id': city_id, 'name': name})

cities_gismeteo

[{'city_id': '3995', 'name': 'Байкит'},
 {'city_id': '158140', 'name': 'Бурный'},
 {'city_id': '4036', 'name': 'Ванавара'},
 {'city_id': '157875', 'name': 'Вельмо'},
 {'city_id': '158142', 'name': 'Кислокан'},
 {'city_id': '158143', 'name': 'Кузьмовка'},
 {'city_id': '158146', 'name': 'Муторай'},
 {'city_id': '157247', 'name': 'Преображенка'},
 {'city_id': '156730', 'name': 'Преображенка'},
 {'city_id': '158152', 'name': 'Суломай'},
 {'city_id': '157881', 'name': 'Тея'},
 {'city_id': '4015', 'name': 'Тура'},
 {'city_id': '158154', 'name': 'Учами'},
 {'city_id': '158155', 'name': 'Чемдальск'}]

В списке есть дубликат населённого пункта Преображенка.

### Реализация проверки наличия метео-данных

Получение исторических метео-данных происходит следующим образом:
1. Задаются все необходимые параметры - страна, область, город, месяц и год
2. Нажатие на кнопку Получить дневник выполняет перенаправление на url вида  
https://www.gismeteo.ru/diary/4036/2008/1/  
в котором:  
- **4036** - ID населённого пункта
- **2008** - год
- **1** - месяц без лидирующего нуля

Если данные погоде не были зафиксированы, то вместо дневника будет сообщение об отсутствии данных.

In [11]:
import os
from bs4 import BeautifulSoup

def get_gismeteo_page(city_id, year, month):
    file_name = os.path.join('gismeteo', str(city_id), f'{year}-{month:02d}.html')
    if is_data_exists(file_name, is_raw=True):
        weather = open_file(file_name, is_raw=True)
    else:
        url = f'https://www.gismeteo.ru/diary/{city_id}/{year}/{month}/'
        r = get_url(url)
        weather = r.text
        write_data(file_name, data=weather, is_raw=True)
    soup = BeautifulSoup(weather, 'lxml')
    if soup.find(class_='empty_phrase'):
        return None
    return soup

In [12]:
print(f'Число пунктов до чистки: {len(cities_gismeteo)}\n')
for city in cities_gismeteo.copy():
    if get_gismeteo_page(city["city_id"], 2008, 1) is None:
        cities_gismeteo.remove(city)
        print(f'{city["name"]} отсутствует')


Число пунктов до чистки: 14

Бурный отсутствует
Вельмо отсутствует
Кислокан отсутствует
Кузьмовка отсутствует
Муторай отсутствует
Преображенка отсутствует
Преображенка отсутствует
Суломай отсутствует
Тея отсутствует
Учами отсутствует
Чемдальск отсутствует

Число пунктов после чистки: 3
р.Подкаменная Тунгуска - с.Байкит
р.Подкаменная Тунгуска - с.Ванавара
р.Нижняя Тунгуска - пгт Тура


In [35]:
print(f'\nЧисло пунктов после чистки: {len(cities_gismeteo)}')
for city in cities_gismeteo:
    print(next(x['clean_name'] for x in posts.values() if city['name'] == x['clean_name']))
    #print(next(x['name'] for x in posts.values() if city['name'] == x['clean_name']))

print(f'\nПункты без информации: {len(names) - len(cities_gismeteo)}')
for name in names:
    if name not in city.values():
        print(name)


Число пунктов после чистки: 3
Байкит
Ванавара
Тура

Пункты без информации: 22
Кислокан
Ванавара
Светлана
Суломай
Стрелка-Чуня
Вельмо
Преображенка
Токма
Чемдальск
Ербогачен
Непа
Тембенчи
Ика
Учами
Бурный
Большой Порог
Усть-Камо
Байкит
Тея
Верхнекарелина
Кербо
Кузьмовка
Подволошино
Муторай


In [13]:
'''
[0]      |             день               |            вечер               |
[1] число|темп.|давл.|облач.|явления|ветер|темп.|давл.|облач.|явления|ветер|
   ------------------------------------------------------------------------
[2]   1  | -15 | 732
   -----------------
'''
def process_row(row):
    cells = row.find_all('td')
    day = int(cells[0].text)
    temperature = int(cells[1].text)
    pressure = int(cells[2].text)
    
    # 3 cloud
    cloud = 'unknown'
    url_cloud = cells[3].find('img')
    if url_cloud is not None:
        url_cloud = url_cloud['src']  # //st6.gismeteo.ru/static/diary/img/dull.png
        cloud = re.search(r"diary/img/(\w+).png", url_cloud).group(1)
        
    # 4 weather
    weather = 'clear'
    url_weather = cells[4].find('img')
    if url_weather is not None:
        url_weather = url_weather['src'] # //st6.gismeteo.ru/static/diary/img/snow.png
        weather = re.search(r"diary/img/(\w+).png", url_cloud).group(1)
    
    # 5 wind
    if cells[5].text == 'Ш':  # штиль
        day_wind_spd, day_wind_dir = 0, 'Ш'
    else:
        wind_info = re.search(r"(\w+) (\d+)", cells[5].text)
        day_wind_dir = wind_info.group(1)
        day_wind_spd = int(wind_info.group(2))
    # в реальных прогнозах погоды даётся 1 прогноз на весь день, без разделения по часам или времени суток
    
    return (day, temperature, pressure, cloud, weather, day_wind_spd, day_wind_dir)
    #print(type(day), type(temperature), type(pressure), type(cloud), type(weather), type(day_wind_spd), type(day_wind_dir))
    

table = soup.find('table')
rows = table.find_all('tr')
rows = rows[2:]  # убрать шапку таблицы

result = process_row(rows[2])
result

NameError: name 'soup' is not defined

In [ ]:
start_year = 2008
end_year = 2018